## Setup

In [1]:
import os
import sys
import random

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
import numpy as np

sys.path.append("../src")

import models  # noqa: E402
import utils  # noqa: E402
import train.backprop  # noqa: E402
import train.ff  # noqa: E402

os.makedirs("../results", exist_ok=True)

In [2]:
# Config

batch_size = 32
n_classes = 10
lr = 0.001
n_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [3]:
# Load data

train_loader, val_loader, test_loader = utils.load_mnist_data(batch_size)
class_names = [str(i) for i in range(n_classes)]

## Backprop

In [ ]:
# Initialize model, loss, optimizer
model = models.LeNet5(n_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

n_params = sum(p.numel() for p in model.parameters())
torchinfo.summary(model, input_size=(batch_size, 1, 32, 32))
print("")

# Train model using backpropagation
history = train.backprop.backprop(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=n_epochs,
    device=device,
)

# Save trained model
save_path = "results/backprop-model.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"\nTraining completed. Model saved to '{save_path}'")

# Save training history
dst = "results/backprop-history.csv"
pd.DataFrame(history).to_csv(dst, index=False)

# Evaluate model
test_metrics = train.backprop.evaluate(model, test_loader, criterion, device)
utils.save_metrics(test_metrics, "results/mnist-metrics.csv", "backprop")

# Final summary
print("\nBackpropagation – Training Summary")
print("-" * 60)
print(f"Train accuracy: {history['train_accuracies'][-1]:.2f}%")
print(f"Val accuracy: {history['val_accuracies'][-1]:.2f}%")
print(f"Test accuracy: {test_metrics['accuracy']:.2f}%")
print(f"Params: {n_params:,}")
print(f"Epochs: {n_epochs}")

## Forward-forward

In [ ]:
# Initialize model, loss, optimizer
model = models.FFLeNet5(n_classes=n_classes)
optimizers = [torch.optim.Adam(layer.parameters(), lr=lr) for layer in model.layers]

n_params = sum(p.numel() for p in model.parameters())
torchinfo.summary(model, input_size=(batch_size, 1 + n_classes, 32, 32))
print("")

# Train model using forward-forward
# - No criterion: FF does not use global loss
# - No optimizer: FF uses per-layer optimizers
history = train.ff.forward_forward(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizers=optimizers,
    n_epochs=n_epochs,
    device=device,
    n_classes=n_classes,
    threshold=2.0,
)

# Save trained model
save_path = "results/ff-model.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"\nTraining completed. Model saved to '{save_path}'")

# Save training history
dst = "results/ff-history.csv"
pd.DataFrame(history).to_csv(dst, index=False)

# Evaluate model
test_metrics = train.ff.evaluate(model, test_loader, n_classes, device)
utils.save_metrics(test_metrics, "results/mnist-metrics.csv", "ff")

# Final summary
print("\nForward-Forward – Training Summary")
print("-" * 60)
if history["train_accuracies"][-1] is not None:
    print(f"Train accuracy: {history['train_accuracies'][-1]:.2f}%")
print(f"Val accuracy: {history['val_accuracies'][-1]:.2f}%")
print(f"Test accuracy: {test_metrics['accuracy']:.2f}%")
print(f"Params: {n_params:,}")
print(f"Epochs: {n_epochs}")


Starting forward-forward training for 25 epochs
------------------------------------------------------------
Train Acc: N/A,  Val Acc:   0.79%,  Time: 33.80s
Train Acc: N/A,  Val Acc:   0.84%,  Time: 32.79s
Train Acc: N/A,  Val Acc:   0.90%,  Time: 33.21s
Train Acc: N/A,  Val Acc:   0.87%,  Time: 33.64s
Train Acc: N/A,  Val Acc:   0.90%,  Time: 33.26s
Train Acc: N/A,  Val Acc:   0.81%,  Time: 32.81s
Train Acc: N/A,  Val Acc:   0.87%,  Time: 34.88s
Train Acc: N/A,  Val Acc:   0.91%,  Time: 38.25s
Train Acc: N/A,  Val Acc:   0.92%,  Time: 40.16s
Train Acc: N/A,  Val Acc:   0.91%,  Time: 40.05s
Train Acc: N/A,  Val Acc:   0.93%,  Time: 37.87s
Train Acc: N/A,  Val Acc:   0.92%,  Time: 35.62s
Train Acc: N/A,  Val Acc:   0.93%,  Time: 37.30s
Train Acc: N/A,  Val Acc:   0.92%,  Time: 36.60s
Train Acc: N/A,  Val Acc:   0.93%,  Time: 37.28s
Train Acc: N/A,  Val Acc:   0.92%,  Time: 37.55s
Train Acc: N/A,  Val Acc:   0.92%,  Time: 38.88s
Train Acc: N/A,  Val Acc:   0.93%,  Time: 37.93s
Train Ac

TypeError: unsupported format string passed to NoneType.__format__

## Results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


def plot_training_curves(
    histories: dict[str, pd.DataFrame],
    metrics: list[str] = ["losses"],
    save_path: str = None,
    title: str = "Training Curves",
):
    plt.figure(figsize=(10, 6))

    colors = plt.get_cmap("tab10")
    model_colors = {model_name: colors(i) for i, model_name in enumerate(histories)}

    line_styles = {"train": "-", "val": "--"}

    for model_name, df in histories.items():
        for metric in metrics:
            for phase in ["train", "val"]:
                col = f"{phase}_{metric}"
                if col in df.columns:
                    plt.plot(
                        df[col],
                        label=f"{model_name} ({phase})",
                        color=model_colors[model_name],
                        linestyle=line_styles[phase],
                    )

    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Saved plot to {save_path}")
    else:
        plt.show()


def plot_test_metrics(csv_path: str, save_path: str = None):
    """
    Plot bar chart of test metrics from CSV.

    Args:
        csv_path: Path to mnist-metrics.csv
        save_path: If set, saves the figure instead of showing it.
    """
    df = pd.read_csv(csv_path)
    metric_cols = ["accuracy", "precision", "recall", "f1_score"]

    df_plot = df.set_index("model")[metric_cols]

    df_plot.plot(kind="bar", figsize=(10, 5))
    plt.title("Final Test Performance")
    plt.ylabel("Score")
    plt.ylim(0, 1.0)
    plt.grid(True, axis="y")
    plt.xticks(rotation=0)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Saved test performance plot to {save_path}")
    else:
        plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


def plot_learning_rule_performance(
    histories: dict[str, pd.DataFrame], save_path: str = None
):
    """
    Plot training/validation loss and accuracy + average epoch time barplot.

    Parameters:
        histories: dict mapping method name to a pd.DataFrame with columns like:
            - train_loss
            - train_accuracy
            - val_loss
            - val_accuracy
            - epoch_times
        save_path: optional path to save the figure
    """
    methods = list(histories.keys())
    colors = {m: c for m, c in zip(methods, sns.color_palette("tab10", len(methods)))}

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    metrics = ["loss", "accuracy"]
    phases = ["train", "val"]

    for row, phase in enumerate(phases):
        for col, metric in enumerate(metrics):
            ax = axes[row, col]
            for method in methods:
                df = histories[method]
                col_name = f"{phase}_{metric}"

                if col_name not in df.columns:
                    continue

                ax.plot(
                    df[col_name],
                    label=method,
                    color=colors[method],
                    linestyle="-" if phase == "train" else "--",
                )

            ax.set_title(f"{phase.capitalize()} {metric.capitalize()}")
            ax.set_xlabel("Epoch")
            ax.set_ylabel(metric.capitalize())
            ax.grid(True)

    # Compute average epoch time
    avg_times = {}
    for method, df in histories.items():
        if "epoch_times" in df.columns:
            avg_times[method] = df["epoch_times"].mean()

    time_df = pd.DataFrame(
        {
            "Method": list(avg_times.keys()),
            "Avg Epoch Time (s)": list(avg_times.values()),
        }
    )

    # Right-side bar plot
    from matplotlib.gridspec import GridSpec

    fig.subplots_adjust(right=0.75)  # Leave room on right
    gs = GridSpec(2, 3, figure=fig)
    bar_ax = fig.add_subplot(gs[:, 2])  # Spans both rows

    sns.barplot(
        data=time_df,
        y="Method",
        x="Avg Epoch Time (s)",
        ax=bar_ax,
        palette=[colors[m] for m in time_df["Method"]],
    )
    bar_ax.set_title("Avg Epoch Time")
    bar_ax.grid(True, axis="x")

    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper center", ncol=len(methods))

    plt.tight_layout(rect=[0, 0, 0.75, 0.95])
    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

In [ ]:
history_backprop = pd.read_csv("results/backprop-history.csv")
history_ff = pd.read_csv("results/ff-history.csv")

plot_training_curves(
    {"backprop": history_backprop, "ff": history_ff},
    save_path="../results/training-curves.png",
)
# plot_test_metrics("mnist-metrics.csv", save_path="../results/test-performance.png")